In [16]:
# import required modules
# Используем dask, распараллеллим вычисления, pandas не вытягивает такой объем (только если частями выкачивать)
from dask import dataframe as dd
dask_df = dd.read_csv('data/train.csv')

In [17]:
# Отдельно выберем лекции и поработаем с ними
dask_df_l = dask_df.loc[dask_df["content_type_id"] == 1].compute()
# Удалим не нужные нас столбцы row_id, content_type_id (в датафрейме только лекции), 
# user_answer, answered_correctly, prior_question_elapsed_time, prior_question_had_explanation
dask_df_l = dask_df_l.drop('row_id', axis=1)
dask_df_l = dask_df_l.drop('timestamp', axis=1)
dask_df_l = dask_df_l.drop('content_type_id', axis=1)
dask_df_l = dask_df_l.drop('user_answer', axis=1)
dask_df_l = dask_df_l.drop('answered_correctly', axis=1)
dask_df_l = dask_df_l.drop('prior_question_elapsed_time', axis=1)
dask_df_l = dask_df_l.drop('prior_question_had_explanation', axis=1)

C:\Users\a_l_b\anaconda3\lib\site-packages\dask\dataframe\io\csv.py:181: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [18]:
# Выгрузим лекции в датафрейм
#lectures = dd.read_csv('data/lectures.csv')
import pandas as pd
lectures = pd.read_csv('data/lectures.csv')
display(lectures.head())

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [19]:
# Переименуем название поля lecture_id в content_id, создав копию с другим именем и удалив старый(это нужно чтоб соединить по content_id)
lectures['content_id'] = lectures['lecture_id']
lectures = lectures.drop('lecture_id', axis=1)
# Соединим два датафрейма
lectures_join = dask_df_l.merge(lectures, how="left", on=["content_id"])

In [21]:
# Сделаем отдельный датасет со спискол студентов ходивших на лекции чтобы поставить флаг ходил или нет
df_unique_lectures = pd.DataFrame(list(lectures_join['user_id'].unique()))
df_unique_lectures['user_id'] = df_unique_lectures[0]
df_unique_lectures = df_unique_lectures.drop(0, axis=1)
df_unique_lectures['lec'] = 1
display(df_unique_lectures)

,user_id,lec
0,2746,1
1,5382,1
2,8623,1
3,12741,1
4,13134,1
...,...,...
149601,2147419988,1
149602,2147469944,1
149603,2147470770,1
149604,2147470777,1


In [7]:
#Соберем статистику ответов
dask_df_s = dask_df.loc[dask_df["content_type_id"] != 1].compute()

C:\Users\a_l_b\anaconda3\lib\site-packages\dask\dataframe\io\csv.py:181: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [8]:
dask_df_s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99271300 entries, 0 to 1080610
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 8.1+ GB


In [12]:
# удалим ненужные столбцы
dask_df_s = dask_df_s.drop('row_id', axis=1)
dask_df_s = dask_df_s.drop('timestamp', axis=1)
dask_df_s = dask_df_s.drop('content_type_id', axis=1) # Лекции убрали, столбец стал ненужным
dask_df_s = dask_df_s.drop('user_answer', axis=1) # Есть поле верный/неверный ответ, поэтому сам ответ можно получить из таблице вопросов
# преобразуем питы чтобы уменьшить место
dask_df_s['prior_question_elapsed_time'] = dask_df_s['prior_question_elapsed_time'].astype('category')

In [13]:
dask_df_s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99271300 entries, 0 to 1080610
Data columns (total 6 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   user_id                         int64   
 1   content_id                      int64   
 2   task_container_id               int64   
 3   answered_correctly              int64   
 4   prior_question_elapsed_time     category
 5   prior_question_had_explanation  object  
dtypes: category(1), int64(4), object(1)
memory usage: 4.6+ GB


In [22]:
# Соединим два датафрейма - студетов которые ходят на лекции df_unique_lectures и стстистику ответов поставив признак lec
dask_df_s_join = dask_df_s.merge(df_unique_lectures, how="left", on=["user_id"])

In [30]:
dask_df_s_join['lec'] = dask_df_s_join['lec'].astype('category')

In [31]:
dask_df_s_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99271300 entries, 0 to 99271299
Data columns (total 7 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   user_id                         int64   
 1   content_id                      int64   
 2   task_container_id               int64   
 3   answered_correctly              int64   
 4   prior_question_elapsed_time     category
 5   prior_question_had_explanation  object  
 6   lec                             category
dtypes: category(2), int64(4), object(1)
memory usage: 4.7+ GB


In [ ]:
# сохраним датафрейм и достанем его заново
dask_df_s_join.to_csv('data/dask_df_s_join.csv')

In [3]:
from dask import dataframe as dd
dask_df_s_join = dd.read_csv('data/dask_df_s_join.csv')

In [4]:
# Отдельно студентов которые ходят на лекции и поработаем с ними
dask_df_stud_wits_lec = dask_df_s_join.loc[dask_df_s_join["lec"] == 1].compute()

In [5]:
# Отдельно студентов которые НЕ ходят на лекции и поработаем с ними
dask_df_stud_wits_out_lec = dask_df_s_join.loc[dask_df_s_join["lec"] != 1].compute()

In [6]:
dask_df_stud_wits_lec['answered_correctly'].mean()
# У тех кто ходит на лекции процент правильных ответов 66.58%

0.6657984511210198

In [7]:
dask_df_stud_wits_out_lec['answered_correctly'].mean()
# У тех кто не ходит на лекции процент правильных ответов 57.21%

0.5721048079369118

In [8]:
dask_df_stud_wits_lec['prior_question_elapsed_time'].mean()
# У тех кто ходит на лекции 

25706.602822882363

In [9]:
dask_df_stud_wits_out_lec['prior_question_elapsed_time'].mean()
# У тех кто не ходит на лекции 

23669.0450244644